In [2]:
import typing

import matplotlib.pyplot as plt
import numpy as np
from scipy import special

import sys
sys.path.append("..")

In [ ]:
def current_loop_vector_potential(
    positions: np.ndarray,
    *,
    loop_center: typing.Sequence[float] = (0, 0, 0),
    loop_radius: float = 1e-6,
    current: float = 1e-3,
):
    """Calculates the magnetic vector potential [Ax, Ay] at ``positions``
    due to a 1D current loop.
    Input units are meters and Amperes, output units are Tesla * meter.
    Args:
        positions: Shape (n, 3) array of (x, y, z) positions at which to
            evaluate the vector potential.
        loop_center: (x, y, z) coordinates of the current loop center.
        loop_radius: radius of the current loop.
        current: Magnitude of the current flowing in the loop.
    Returns:
        Shape (n, 3) array of the vector potential [Ax, Ay, Az] at ``positions``.
    """
    # http://www.physics.usu.edu/Wheeler/EMarchive/Jch5Notes.pdf
    positions = np.atleast_2d(positions)
    loop_center = np.atleast_2d(loop_center)
    a = loop_radius

    positions = positions - loop_center
    # This is a pint-friendly vector norm.
    rs = np.sqrt(np.sum(np.square(positions), axis=1))
    # rs = la.norm(positions, axis=1)
    thetas = np.arccos(positions[:, 2] / rs)
    sin_thetas = np.sin(thetas)
    # m == k**2, see docs for scipy.special.ellipk
    denom = rs**2 + a**2 + 2 * a * rs * sin_thetas
    m = 4 * a * rs * sin_thetas / denom
    K = special.ellipk(m)
    E = special.ellipe(m)
    mag = -mu_0 * current * a / (np.pi * m) * (((m - 2) * K + 2 * E)) / np.sqrt(denom)
    # \vec{A} is directed along the azimuthal direction,
    # so here we generate the azimuthal unit vector.
    # Azimuthal angle + pi / 2 to get azimuthal direction.
    phis = np.arctan2(positions[:, 1], positions[:, 0]) + np.pi / 2
    direc = np.stack([np.cos(phis), np.sin(phis), np.zeros_like(phis)], axis=1)
    return mag[:, np.newaxis] * direc

In [ ]:
import tdgl

In [ ]:
mu_0 = tdgl.units.ureg("mu_0").to_base_units().m

In [ ]:
layer = tdgl.device.components.Layer(
    "base", coherence_length=1, london_lambda=100, thickness=0.1, z0=0,
)

In [ ]:
film = tdgl.device.components.Polygon("film", layer=layer.name, points=tdgl.geometry.box(50, points_per_side=80))
source = tdgl.device.components.Polygon(points=tdgl.geometry.box(0.1, 51, center=(-25, 0)))
drain = source.scale(xfact=-1)

In [ ]:
device = tdgl.device.device.Device(
    "box",
    layer=layer,
    film=film,
    source_terminal=source,
    drain_terminal=drain,
    length_units="um",
)

In [ ]:
device.make_mesh(min_points=1000, optimesh_steps=100)

In [ ]:
device.plot(mesh=True)

In [ ]:
device.points.shape

In [ ]:
from tdgl.em import uniform_Bz_vector_potential

In [ ]:
ureg = device.ureg
loop_center = np.array([[0, 0, 1]])
loop_radius = 1
loop_current = 1e-3

def applied_potential(x, y, z, field="1 uT"):
    
#     loop_center = np.array([[0, 0, 1]])
#     loop_radius = 2.5
#     loop_current = 1e-4

    length_units = ureg(device.length_units)
    positions = (
        np.stack([x, y, z], axis=1) * length_units
    ).to("m")
#     loop_center = (loop_center * length_units).to("m").m
#     loop_radius = (loop_radius * length_units).to("m").m
#     potential = current_loop_vector_potential(
#         positions,
#         loop_center=loop_center,
#         current=loop_current,
#         loop_radius=loop_radius,
#     ) * ureg("T * m")
    potential = uniform_Bz_vector_potential(positions, field)
    return potential.to(f"mT * {device.length_units}").m

In [ ]:
from tdgl.solve import solve

In [ ]:
vector_potential = tdgl.parameter.Parameter(applied_potential, field="2 uT")

In [ ]:
solution = solve(
    device,
    vector_potential,
    "output1.h5",
    field_units="mT",
    gamma=10,
    dt=1e-2,
    skip=0,
    steps=10_000,
    source_drain_current=0,
)

In [ ]:
xs = np.linspace(-25 / 1, 25 / 1, 101)
cross = [
    np.stack([xs, xs], axis=1)
]

_ = solution.plot_currents(cross_section_coords=cross, dataset="supercurrent", figsize=(4, 6), streamplot=True)

In [ ]:
f = solution.polygon_fluxoid(tdgl.geometry.circle(5, center=(0, 0), points=101))

In [ ]:
f

In [ ]:
f.flux_part / f.supercurrent_part

In [ ]:
import superscreen as sc

In [ ]:
device = sc.Device(
    "box",
    layers=[sc.Layer("base", london_lambda=50, thickness=0.1, z0=0)],
    films=[sc.Polygon("film", layer="base", points=tdgl.geometry.box(50 / 2))],
    abstract_regions=[
        sc.Polygon("bounding_box", layer="base", points=tdgl.geometry.box(55 / 2)),
    ],
    length_units="um"
)

In [ ]:
device.make_mesh(min_points=2000, optimesh_steps=20)

In [ ]:
from tdgl.em import current_loop_field

In [ ]:
def applied_field(x, y, z, *, current, loop_center, loop_radius):
    if z.shape != x.shape:
        z = z[0] * np.ones_like(x)
    positions = np.stack([x, y, z], axis=1) * 1e-6
    field_in_tesla = current_loop_field(
        positions,
        loop_center=np.array(loop_center) * 1e-6,
        loop_radius=loop_radius * 1e-6,
        current=current
    )
    return 1e3 * field_in_tesla[:, 2]

In [ ]:
solution = sc.solve(
    device,
#     applied_field=sc.Parameter(
#         applied_field,
#         current=1e-4,
#         loop_center=(0, 0, 1),
#         loop_radius=2.5,
#     ),
    applied_field=sc.sources.ConstantField(2e-3),
    field_units="mT",
    current_units="uA",
)[-1]

In [ ]:
solution.polygon_fluxoid(tdgl.geometry.circle(5, center=(0, 0), points=101))

In [ ]:
xs = np.linspace(-25, 25, 101)
cross = [
    np.stack([xs, xs], axis=1)
]

_ = solution.plot_currents(cross_section_coords=cross, grid_method="cubic", figsize=(4, 6), streamplot=False)

In [ ]:
xs = np.linspace(-25, 25, 101)
cross = [
    np.stack([xs, xs], axis=1)
]

_ = solution.plot_fields(cross_section_coords=cross, figsize=(4, 6), units="uT")

In [ ]:
solution.polygon_fluxoid(tdgl.geometry.circle(5, center=(0, 0), points=101))

In [ ]:
solution

In [ ]:
(-0.03737217 / -0.00966151097)**2

In [ ]:
-0.11848215 / -0.0313327807

In [ ]:
np.linalg.norm(solution.current_densities["base"], axis=1).max()

In [ ]:
xy = device.points
zs = 0

A = solution.applied_vector_potential(xy[:, 0], xy[:, 1], np.zeros(xy.shape[0]))

In [ ]:
A

In [3]:
from superscreen import ureg

In [11]:
sigma = 1 / ureg("1.5e-7 ohm * m")
lamda = ureg("1 um")
mu_0 = ureg("mu_0")
tau = (sigma * mu_0 * lamda**2).to("s")

In [14]:
tau.to("ns")

0.008377580414167066 <Unit('nanosecond')>